In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import json
import vtk
from dataloading import loader
from vtk.util import numpy_support
from vedo import Mesh, Plotter, show, settings, Points, Point, Plane
import os
import numpy as np
import pandas as pd
import miniball
from typing import List
from processing import meshes as meshes_processing

settings.default_backend = "k3d"

In [14]:
mesh_filename = "data/lower/O52P1SZT/O52P1SZT_lower.obj"
labels_filename = "data/lower/O52P1SZT/O52P1SZT_lower.json"
# Loading the data
with open(labels_filename, "r") as file:
    labels = json.load(file)
polydata = loader.read_obj(mesh_filename)
# Check number of points/labels/instances
polydata_points = numpy_support.vtk_to_numpy(polydata.GetPoints().GetData())

assert len(polydata_points) == len(labels["labels"])
assert len(polydata_points) == len(labels["instances"])

In [15]:
np.unique(labels["labels"])

array([ 0, 31, 32, 33, 34, 35, 36, 41, 42, 43, 44, 45, 46, 47])

In [16]:
# checking the label of the wisdom tooth
LOWER_PATH = "data/lower"
folders = os.listdir(LOWER_PATH)
obj_files = [os.path.join(LOWER_PATH, f, f"{f}_lower.obj") for f in folders]
label_files = [os.path.join(LOWER_PATH, f, f"{f}_lower.json") for f in folders]
assert len(obj_files) == len(label_files)

In [17]:
data = []
for i, (obj_file, label_file) in enumerate(zip(obj_files, label_files)):
    with open(label_file, "r") as file:
        labels = json.load(file)
    data.append([obj_file, np.array(labels["labels"]), np.unique(labels["labels"])])

# Create a DataFrame from the list
columns = ["obj_file", "labels", "unique_labels"]
df = pd.DataFrame(data, columns=columns)
df["num_teeth"] = df["unique_labels"].apply(len)
filtered_df = df[df["num_teeth"] >= 17]

In [30]:
# for obj_file, label_file in zip(obj_files[:2], label_files[:2]):

obj_file = filtered_df.iloc[3].obj_file
labels = filtered_df.iloc[3].labels
polydata = loader.read_obj(obj_file)

label = labels
print(np.unique(label))
print(len(np.unique(label)))

label = np.zeros(np.array(labels).shape)
print(np.unique(labels))
labels = np.array(labels).astype(int)

label[labels == 37] = 1
print(np.unique(label))

# Add the objects to the plotter
# Convert the VTK PolyData to a vedo Mesh
vedo_mesh = Mesh(polydata)
vedo_mesh.cmap("jet", label, on="points")

# Show the plot
show(vedo_mesh)

[ 0 31 32 33 34 35 36 37 38 41 42 43 44 45 46 47 48]
17
[ 0 31 32 33 34 35 36 37 38 41 42 43 44 45 46 47 48]
[0. 1.]


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [31]:
label = label
polydata = polydata
polydata_points = numpy_support.vtk_to_numpy(polydata.GetPoints().GetData())
tooth_to_extract = polydata_points[label == 1]
tooth_df = pd.DataFrame(tooth_to_extract, columns=["x", "y", "z"])
sphere_center, sphere_radius = miniball.get_bounding_ball(tooth_df.values)

In [32]:
sphere_source = meshes_processing.define_sphere_source(sphere_center, sphere_radius / 6)
sphere_polydata = sphere_source.GetOutput()
Plotter()
# Create a PointCloud object
point_cloud = Points(tooth_to_extract)
vedo_sphere = Mesh(sphere_polydata)

# Show the point cloud
show(vedo_sphere, vedo_mesh)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [33]:
plane = meshes_processing.define_plane(sphere_center, [0, 0, 1])

plane_vedo = Plane(sphere_center, normal=(0, 0, 1), s=(10, 10))

bounds = [
    sphere_center[0] - 5,
    sphere_center[0] + 5,
    sphere_center[1] - 5,
    sphere_center[1] + 5,
    sphere_center[2] - 5,
    sphere_center[2] + 5,
]

cut_polydata = meshes_processing.cut_by_bounds(polydata, bounds)
cut_vedo_mesh = Mesh(cut_polydata, "blue")
Plotter()
show(cut_vedo_mesh)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…